# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [2]:
def european_put(P, K, r, f, rf):
    P1up = P * (1+r)
    P1down = P * (1+f)
    P2upup = P1up * (1+r)
    P2updown = P1up * (1+f)
    P2downup = P1down * (1+r)
    P2downdown = P1down * (1+f)
    prob = ((1+rf) ** 0.5 - 1 - f)/(r-f)
    val1 = (prob * max(K - P2upup,0) + (1-prob) * max(K-P2downdown, 0)) / ((1+rf) ** 0.5)
    val2 = (prob * 0 + (1-prob) * max(val1, 0)) / ((1+rf) ** 0.5)
    return val2
            
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...


Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [3]:
def american_put(P, K, r, f, rf):
    P1up = P * (1+r)
    P1down = P * (1+f)
    prob = ((1+rf) ** 0.5 - 1 - f)/(r-f)
    val1 = (prob * max(K-P1up,0) + (1-prob) * max(K-P1down, 0)) / ((1+rf) ** 0.5)
    return val1

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...


### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [4]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

In [5]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    t = 1/12
    cols = random_vars.columns.values.tolist()
    random_prices_llst = []
    for col in cols:
        col = int(col)
        random_prices_lst = []
        for i in random_vars.index:
            if (col == 0):
                random_prices_lst.append(S0 * np.exp((rf - (sigma **2)/2) * t + sigma * random_vars.iloc[i,col] * (t **0.5)))
            else:
                random_prices_lst.append(random_prices_llst[-1][i] * np.exp((rf - (sigma **2)/2) * t + sigma * random_vars.iloc[i,col] * (t **0.5)))
        random_prices_llst.append(random_prices_lst)
    random_prices_llst = np.array(random_prices_llst)
    df = pd.DataFrame(data=random_prices_llst.transpose(),columns = [1,2,3,4,5,6,7,8,9,10,11,12])
    df[0] = S0
    df = df[[0,1,2,3,4,5,6,7,8,9,10,11,12]]
    return df

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...


•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [6]:
def payoff_table(paths, K, T, rf, B):
    european_calls = []
    european_puts = []
    avg_price_put = []
    floating_lookback = []
    up_out = []
    up_in = []
    for i in paths.index:
        S12 = paths.loc[i,12]
        european_calls.append(max(0,S12 - K) * np.exp(-rf * T))
        european_puts.append(max(0,K - S12) * np.exp(-rf * T))
        S_avg = (paths.loc[i,12] + paths.loc[i,11] + paths.loc[i, 10] + paths.loc[i, '9'] + paths.loc[i, '8'] + paths.loc[i, '7'] + paths.loc[i, '6'] + paths.loc[i, '5'] + paths.loc[i, '4'] + paths.loc[i, '3'] + paths.loc[i, '2'] + paths.loc[i, '1'] + paths.loc[i,'0'])/len(paths.columns.values.tolist())
        avg_price_put.append(max(0,K - S_avg) * np.exp(-rf * T))
        S_min = min(paths.loc[i,'12'],paths.loc[i,'11'], paths.loc[i, '10'], paths.loc[i, '9'],paths.loc[i, '8'], paths.loc[i, '7'], paths.loc[i, '6'], paths.loc[i, '5'], paths.loc[i, '4'], paths.loc[i, '3'] , paths.loc[i, '2'] , paths.loc[i, '1'] , paths.loc[i,'0'])
        floating_lookback.append((S12 - S_min) * np.exp(-rf * T))
        
        up_out_condition = True
        up_in_condition = False
        for j in paths.columns.values.tolist():
            if (paths.loc[i,j] >= B):
                up_out_condition = False
                up_in_condition = True
        if (up_out_condition):
            up_out.append(max(0,S12-K) * np.exp(-rf * T))
        else:
            up_out.append(0)
            
        if (up_in_condition):
            up_in.append(max(0,S12-K) * np.exp(-rf * T))
        else:
            up_in.append(0)
        
        
    answer = pd.DataFrame(data = {
        "European Call" : european_calls,
        "European Put" : european_puts,
        "Average Price Put" : avg_price_put,
        "Floating Lookback" : floating_lookback,
        "Up-and-out" : up_out,
        "Up-and-in" : up_in
    })
    
    return answer
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...


Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [7]:
def statistics(payoffs):
    
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.
    means = []
    standard_errs = []
    low_bound_95 = []
    high_bound_95 = []
    N = len(payoffs.index)
    for i in payoffs.columns.values.tolist():
        mean = payoffs[i].mean()
        stdev = payoffs[i].std()
        means.append(mean)
        standard_errs.append(stdev/(N ** 0.5))
        low_bound_95.append(mean - 1.96 * (stdev/(N ** 0.5)))
        high_bound_95.append(mean + 1.96 * (stdev/(N ** 0.5)))
    output = pd.DataFrame(data = {
        "Mean" : means,
        "Standard Error" : standard_errs,
        "Lower Bound" : low_bound_95,
        "Upper Bound" : high_bound_95
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output
# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [8]:
def create_negative_paths(random_vars, S0, rf, sigma):
    t = 1/12
    random_vars = random_vars * -1
    cols = random_vars.columns.values.tolist()
    random_prices_llst = []
    for col in cols:
        col = int(col)
        random_prices_lst = []
        for i in random_vars.index:
            if (col == 0):
                random_prices_lst.append(S0 * np.exp((rf - (sigma **2)/2) * t + sigma * random_vars.iloc[i,col] * (t **0.5)))
            else:
                random_prices_lst.append(random_prices_llst[-1][i] * np.exp((rf - (sigma **2)/2) * t + sigma * random_vars.iloc[i,col] * (t **0.5)))
        random_prices_llst.append(random_prices_lst)
    random_prices_llst = np.array(random_prices_llst)
    df = pd.DataFrame(data=random_prices_llst.transpose(),columns = ['1','2','3','4','5','6','7','8','9','10','11','12'])
    df['0'] = S0
    df = df[['0','1','2','3','4','5','6','7','8','9','10','11','12']]
    return df



Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [9]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    pos_table = payoff_table(create_paths(random_vars, S0, rf, sigma), K, T, rf, B)
    neg_table = payoff_table(create_negative_paths(random_vars, S0, rf, sigma), K, T, rf, B)
    avg_table = (pos_table + neg_table) / 2
    stats = statistics(avg_table)
    return stats


In [12]:
antithetic_statistics(test_r_vars, 72.25, 0.04, 0.35,  70, 1, 90)

Mean  Standard Error  Lower Bound  Upper Bound
European Call      15.020597        2.731435     9.666984    20.374210
European Put        8.598197        1.233336     6.180859    11.015536
Average Price Put   4.032634        0.712479     2.636175     5.429093
Floating Lookback  18.504190        2.416689    13.767480    23.240901
Up-and-out          1.073261        0.475799     0.140695     2.005827
Up-and-in          13.947336        2.968173     8.129717    19.764956

In [11]:
def correct_create_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*randomVariables[j][i])
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_payoff_table(paths, K, T, rf, B):
    N = len(paths)
    #discounts a payment by a year per the question specifications
    def discount(pmt):
        return pmt*np.exp(-rf * T)

    #functions that calculate payoff for each option given a price path
    def EuropeanCall(path):
        return discount(max(path[12] - K, 0))

    def EuropeanPut(path):
        return discount(max(K-path[12], 0))
    
    def AveragePricePut(path):
        Sbar = path.mean()
        return discount(max(K - Sbar, 0))
    
    def FloatingLookback(path):
        Smin = path.min()
        return discount(path[12] - Smin)
    
    def upAndOut(path, B):
        for i in range(13):
            if(path[i] >= B):
                return 0
        return discount(max(path[12] - K, 0))
    
    def upAndIn(path, B):
        atLeastOne = False
        for i in range(13):
            if(path[i] >= B):
                atLeastOne = True
                break
        if atLeastOne:
            St = path[12]
            return discount(max(St - K, 0))
        else:
            return 0
    
    payoffsArr = np.zeros((N, 6))
    
    for i in range(N):
        row = paths.iloc[i]
        payoffsArr[i][0] = EuropeanCall(row)
        payoffsArr[i][1] = EuropeanPut(row)
        payoffsArr[i][2] = AveragePricePut(row)
        payoffsArr[i][3] = FloatingLookback(row)
        payoffsArr[i][4] = upAndOut(row, 90)
        payoffsArr[i][5] = upAndIn(row, 90)
        
    payoffs = pd.DataFrame(payoffsArr)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    payoffs = payoffs.rename(columns=renameDict, errors="raise")
    return payoffs

def correct_statistics(payoffs):
    N = len(payoffs)
    #Statistics
    statLst = []
    for column in payoffs:
        row = []
        info = payoffs[column]
        mean = info.mean()
        sd = info.std()
        se = sd/(N**0.5)
        row.append(mean)
        row.append(se)
        row.append(mean-1.96*se)
        row.append(mean+1.96*se)
        statLst.append(row)

    stats = pd.DataFrame(statLst)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    stats = stats.rename(index=renameDict, errors="raise")
    renameDict2 = {0: "Mean", 1: "Standard Error", 2: "Lower Bound", 3: "Upper Bound"}
    stats = stats.rename(columns=renameDict2, errors="raise")
    return stats



def correct_create_negative_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*(-randomVariables[j][i]))
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    paths1 = correct_create_paths(random_vars, S0, rf, sigma)
    paths2 = correct_create_negative_paths(random_vars, S0, rf, sigma)
    payoffs1 = correct_payoff_table(paths1, K, T, rf, B)
    payoffs2 = correct_payoff_table(paths2, K, T, rf, B)
    
    avg_payoffs = (payoffs1 + payoffs2)/2
    return correct_statistics(avg_payoffs)

test_r_vars = gen_random_vars(700, 20)

test_paths = correct_create_paths(test_r_vars, 72.25, 0.04 , 0.35)

test_payoff_table = correct_payoff_table(test_paths, 70, 1, 0.04, 90)

In [13]:
payoff_table(test_paths, 70, 1, 0.04, 90)

KeyError: '12'